# Capítulo 7 - Decoradores de função e closures

### Básico sobre decoradores

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [2]:
@deco
def target():
    print('running target()')

In [3]:
target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner()>

### Quando Python executa os decoradores

In [5]:
registry = []

In [6]:
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

In [7]:
@register
def f1():
    print('running f1()')

running register(<function f1 at 0x7fdd4e7033a0>)


In [8]:
@register
def f2():
    print('running f2()')

running register(<function f2 at 0x7fdd4e703430>)


In [9]:
def f3():
    print('running f3()')

In [10]:
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

In [11]:
main()

running main()
registry -> [<function f1 at 0x7fdd4e7033a0>, <function f2 at 0x7fdd4e703430>]
running f1()
running f2()
running f3()


### Padrão Strategy melhorado com decorador

In [12]:
promos = []

In [14]:
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

In [15]:
@promotion
def fidelity(order):
    """5% de desconto para clientes com mil ou mais pontos no programa de fidelidade"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [17]:
@promotion
def bulk_item(order):
    """10% de desconto para cada LineItem com 20 ou mais unidades"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

In [18]:
@promotion
def large_order(order):
    """7% de desconto para pedidos com 10 ou mais itens diferentes"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [22]:
def best_promo(order):
    """Seleciona o melhor desconto disponível"""
    return max(promo(order) for promo in promos)

In [23]:
promos

[<function __main__.fidelity(order)>,
 <function __main__.bulk_item(order)>,
 <function __main__.large_order(order)>]

### Regras para escopo de variáveis

In [24]:
def f1(a):
    print(a)
    print(b)

In [27]:
# f1(3)

3


NameError: name 'b' is not defined

In [28]:
b = 6

In [29]:
f1(3)

3
6


In [30]:
b = 6

In [31]:
def f2(a):
    print(a)
    print(b)
    b = 9

In [32]:
# f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

### Closures

In [33]:
class Averager():
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [34]:
avg = Averager()

In [35]:
avg(10)

10.0

In [36]:
avg(11)

10.5

In [37]:
avg(12)

11.0

In [38]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [39]:
avg = make_averager()

In [40]:
avg(10)

10.0

In [41]:
avg(11)

10.5

In [42]:
avg(12)

11.0

In [44]:
avg.__code__.co_varnames

('new_value', 'total')

In [46]:
avg.__code__.co_freevars

('series',)

In [47]:
avg.__code__.co_freevars

('series',)

In [48]:
avg.__closure__

(<cell at 0x7fdd5405aac0: list object at 0x7fdd4cea3140>,)

In [49]:
avg.__closure__[0].cell_contents

[10, 11, 12]

### Declaração nonlocal

In [50]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [51]:
avg = make_averager()

In [52]:
# avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [53]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [54]:
avg = make_averager()

In [55]:
avg(10)

10.0

In [56]:
avg(11)

10.5

In [57]:
avg(12)

11.0

### Implementando um decorador simple

In [58]:
import time

In [60]:
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [61]:
@clock
def snooze(seconds):
    time.sleep(seconds)

In [62]:
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

In [63]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.12315324s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000040s] factorial(1) -> 1
[0.00001148s] factorial(2) -> 2
[0.00001775s] factorial(3) -> 6
[0.00002276s] factorial(4) -> 24
[0.00002795s] factorial(5) -> 120
[0.00003432s] factorial(6) -> 720
6! =  720


In [64]:
import functools

In [67]:
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%s' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

### Decoradores da biblioteca-padrão

### Memoização com functools.lru_cache

In [68]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [69]:
fibonacci(6)

[0.00000024s fibonacci(0) -> 0
[0.00000024s fibonacci(1) -> 1
[0.00007200s fibonacci(2) -> 1
[0.00000000s fibonacci(1) -> 1
[0.00000048s fibonacci(0) -> 0
[0.00000024s fibonacci(1) -> 1
[0.00000715s fibonacci(2) -> 1
[0.00001478s fibonacci(3) -> 2
[0.00009489s fibonacci(4) -> 3
[0.00000000s fibonacci(1) -> 1
[0.00000024s fibonacci(0) -> 0
[0.00000000s fibonacci(1) -> 1
[0.00000644s fibonacci(2) -> 1
[0.00001311s fibonacci(3) -> 2
[0.00000024s fibonacci(0) -> 0
[0.00000000s fibonacci(1) -> 1
[0.00000644s fibonacci(2) -> 1
[0.00000000s fibonacci(1) -> 1
[0.00000048s fibonacci(0) -> 0
[0.00000000s fibonacci(1) -> 1
[0.00000715s fibonacci(2) -> 1
[0.00001383s fibonacci(3) -> 2
[0.00002623s fibonacci(4) -> 3
[0.00004578s fibonacci(5) -> 5
[0.00014877s fibonacci(6) -> 8


8

In [70]:
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [71]:
fibonacci(6)

[0.00000048s fibonacci(0) -> 0
[0.00000024s fibonacci(1) -> 1
[0.00004983s fibonacci(2) -> 1
[0.00000072s fibonacci(3) -> 2
[0.00005913s fibonacci(4) -> 3
[0.00000048s fibonacci(5) -> 5
[0.00006795s fibonacci(6) -> 8


8

### Funções genéricas com dispatch simples

In [72]:
import html

In [74]:
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [75]:
from functools import singledispatch
from collections import abc
import numbers
import html

In [76]:
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [78]:
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<vr>\n')
    return '<p>{0}</p>'.format(content)

In [79]:
@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

In [80]:
@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</lin>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

### Decoradores empilhados

### Decoradores parametrizados

In [102]:
registry = []

In [103]:
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

In [104]:
@register
def f1():
    print('running f1()')

running register(<function f1 at 0x7fdd4e7033a0>)


In [105]:
print('running main()')
print('registry ->', registry)
f1()

running main()
registry -> [<function f1 at 0x7fdd4e7033a0>]
running f1()


### Um decorador de registro parametrizado

In [106]:
registry = set()

In [108]:
def register(active=True):
    def decorate(func):
        print('ruuning register(active=%s) -> decorate(%s)'
              % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

In [109]:
@register(active=False)
def f1():
    print('running f1()')

ruuning register(active=False) -> decorate(<function f1 at 0x7fdd2f83ad30>)


In [110]:
@register()
def f2():
    print('running f2()')

ruuning register(active=True) -> decorate(<function f2 at 0x7fdd2f83a3a0>)


In [111]:
def f3():
    print("running f3()")

In [112]:
registry

{<function __main__.f2()>}

In [113]:
register()(f3)

ruuning register(active=True) -> decorate(<function f3 at 0x7fdd4ced7b80>)


<function __main__.f3()>

In [114]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [115]:
register(active=False)(f2)

ruuning register(active=False) -> decorate(<function f2 at 0x7fdd2f83a3a0>)


<function __main__.f2()>

In [116]:
registry

{<function __main__.f3()>}

### Decorador clock parametrizado

In [117]:
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

In [118]:
def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

In [119]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

In [120]:
for i in range(3):
    snooze(.123)

[0.12309933s] snooze(0.123) -> None
[0.12317276s] snooze(0.123) -> None
[0.12315845s] snooze(0.123) -> None


In [121]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

In [122]:
for i in range(3):
    snooze(.123)

snooze: 0.12313461303710938s
snooze: 0.12308335304260254s
snooze: 0.12317657470703125s


In [123]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

In [124]:
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
